In [ ]:
import json

file = json.load(open("json_extraction_dataset_500.json", "r"))
print(file[1])

{'input': "Extract the product information:\n<div class='product'><h2>iPad Air</h2><span class='price'>$1344</span><span class='category'>audio</span><span class='brand'>Dell</span></div>", 'output': {'name': 'iPad Air', 'price': '$1344', 'category': 'audio', 'manufacturer': 'Dell'}}


In [ ]:
# !pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 3 | Total steps = 96
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
25,0.117000
50,0.112800
75,0.108800


In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Extract the product information:\n<div class='product'><h2>iPad Air</h2><span class='price'>$1344</span><span class='category'>audio</span><span class='brand'>Dell</span></div>"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> Extract the product information:
<div class='product'><h2>iPad Air</h2><span class='price'>$1344</span><span class='category'>audio</span><span class='brand'>Dell</span></div><|end|><|assistant|> {"name": "iPad Air", "price": "$1344", "category": "audio", "manufacturer": "Dell"}<|end|>


In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [04:04<00:00, 122.12s/it]


Unsloth: Merge process complete. Saved to `/content/gguf_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['phi-3-mini-4k-instruct.F16.gguf']
Unsloth: [2] Converting GGUF f16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['phi-3-mini-4k-instruct.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: llama-cli --model phi-3-mini-4k-ins

{'save_directory': 'gguf_model',
 'gguf_files': ['phi-3-mini-4k-instruct.Q4_K_M.gguf'],
 'modelfile_location': '/content/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)

In [ ]:
import json
import re

import json
import re

def extract_json_from_response(response):
    """Extract JSON from model response"""
    start = response.rfind('{')
    end = response.rfind('}') + 1
    if start != -1 and end > start:
        return response[start:end]
    return None

FastLanguageModel.for_inference(model)

# REAL TEST CASES - Testing model's actual capabilities
test_cases = [
    {
        "name": "Test 1: Complete Data (Baseline)",
        "html": """<div class='product'><h2>iPad Pro 12.9</h2><span class='price'>$1099</span><span class='category'>tablets</span><span class='brand'>Apple</span></div>""",
        "expected": "Should extract all fields correctly"
    },
    {
        "name": "Test 2: Missing Price",
        "html": """<div class='product'><h2>iPad Pro 12.9</h2><span class='category'>tablets</span><span class='brand'>Apple</span></div>""",
        "expected": "Can it handle missing price?"
    },
    {
        "name": "Test 3: Missing Brand",
        "html": """<div class='product'><h2>Gaming Mouse</h2><span class='price'>$79</span><span class='category'>accessories</span></div>""",
        "expected": "Can it handle missing brand?"
    },
    {
        "name": "Test 4: Different Order of Elements",
        "html": """<div class='product'><span class='brand'>Samsung</span><span class='price'>$1299</span><h2>Galaxy Book Pro</h2><span class='category'>laptops</span></div>""",
        "expected": "Can it handle different element order?"
    },
    {
        "name": "Test 5: Only Product Name",
        "html": """<div class='product'><h2>AirPods Pro</h2></div>""",
        "expected": "What happens with minimal data?"
    },
    {
        "name": "Test 6: New Product (Not in Training)",
        "html": """<div class='product'><h2>Meta Quest 3</h2><span class='price'>$499</span><span class='category'>VR</span><span class='brand'>Meta</span></div>""",
        "expected": "Can it handle products it never saw?"
    },
    {
        "name": "Test 7: Different Format - Extra Spaces",
        "html": """<div class='product'>  <h2>  PlayStation 5  </h2>  <span class='price'>  $499  </span>  <span class='category'>  gaming  </span>  <span class='brand'>  Sony  </span>  </div>""",
        "expected": "Can it handle messy formatting?"
    },
    {
        "name": "Test 8: Wrong Class Names",
        "html": """<div class='item'><h2>Xbox Series X</h2><span class='cost'>$499</span><span class='type'>gaming</span><span class='manufacturer'>Microsoft</span></div>""",
        "expected": "What if HTML uses different class names?"
    },
]

print("🧪 REAL MODEL TESTING - Does Fine-tuning Actually Work?")
print("=" * 100)

results = []

for test in test_cases:
    print(f"\n{test['name']}")
    print(f"Expected: {test['expected']}")
    print(f"Input HTML: {test['html'][:100]}...")

    prompt = f"Extract the product information:\n{test['html']}"
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.1)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    json_str = extract_json_from_response(response)

    if json_str:
        try:
            parsed = json.loads(json_str)
            print(f"✅ Output: {json.dumps(parsed, indent=2)}")
            results.append({"test": test['name'], "success": True, "output": parsed})
        except:
            print(f"❌ Failed to parse JSON: {json_str}")
            results.append({"test": test['name'], "success": False, "raw": json_str})
    else:
        print(f"❌ No JSON found in response")
        print(f"Raw response: {response[:300]}")
        results.append({"test": test['name'], "success": False, "raw": response[:200]})

    print("-" * 100)

# Summary
print("\n\n📊 TEST SUMMARY")
print("=" * 100)
success_count = sum(1 for r in results if r.get('success', False))
print(f"Passed: {success_count}/{len(test_cases)}")
print(f"Success Rate: {(success_count/len(test_cases)*100):.1f}%")

if success_count < len(test_cases):
    print("\n⚠️  FINDINGS:")
    print("The model may only work with:")
    print("  • Exact format from training data")
    print("  • Complete information present")
    print("  • Same HTML structure")
    print("\nThis means fine-tuning might be too narrow. Consider:")
    print("  • Adding more diverse training examples")
    print("  • Including incomplete data in training")
    print("  • Testing different HTML formats")

FastLanguageModel.for_inference(model)

# Function to extract JSON from response
def extract_json(response_text):
    """Extract JSON from model response"""
    # Try to find JSON object in the text
    # Look for pattern: {"name": ...}
    match = re.search(r'\{[^{}]*"name"[^{}]*\}', response_text)
    if match:
        return match.group(0)

    # Alternative: find everything after the last '{'
    if '{' in response_text:
        json_start = response_text.rfind('{')
        json_str = response_text[json_start:]
        # Clean up any trailing text after }
        if '}' in json_str:
            json_end = json_str.find('}') + 1
            return json_str[:json_end]

    return None

# Batch process all scraped products
extracted_products = []

print("🔄 Processing products...\n")

for i, html in enumerate(scraped_products, 1):
    prompt = f"Extract the product information:\n{html}"

    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.1)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    print(f"Product {i} - Raw Response:")
    print(response[:200] + "..." if len(response) > 200 else response)
    print()

    # Extract JSON from response
    json_str = extract_json(response)

    if json_str:
        try:
            product_data = json.loads(json_str)
            extracted_products.append(product_data)
            print(f"✅ Successfully extracted: {product_data['name']}")
        except json.JSONDecodeError as e:
            print(f"❌ JSON parse error: {e}")
            print(f"   Attempted to parse: {json_str}")
    else:
        print(f"❌ Could not find JSON in response")

    print("-" * 80 + "\n")

# Display results
if extracted_products:
    print("=" * 80)
    print("🎉 EXTRACTED PRODUCT DATABASE")
    print("=" * 80)
    for i, product in enumerate(extracted_products, 1):
        print(f"\n{i}. {product['name']}")
        print(f"   💰 Price: {product['price']}")
        print(f"   📦 Category: {product['category']}")
        print(f"   🏭 Brand: {product['manufacturer']}")

    # Save to database/file
    with open("extracted_products.json", "w") as f:
        json.dump(extracted_products, f, indent=2)

    print("\n\n✅ Saved to 'extracted_products.json'")

    # Analytics
    print("\n" + "=" * 80)
    print("📊 ANALYTICS")
    print("=" * 80)
    print(f"Total Products: {len(extracted_products)}")
    print(f"Laptops: {sum(1 for p in extracted_products if p['category'] == 'laptops')}")
    print(f"Smartphones: {sum(1 for p in extracted_products if p['category'] == 'smartphones')}")

    # Fixed average price calculation
    total_price = sum(int(p['price'].replace('$', '').replace(',', '')) for p in extracted_products)
    avg_price = total_price / len(extracted_products)
    print(f"Average Price: ${avg_price:.2f}")
else:
    print("⚠️  No products were successfully extracted!")

🧪 REAL MODEL TESTING - Does Fine-tuning Actually Work?

Test 1: Complete Data (Baseline)
Expected: Should extract all fields correctly
Input HTML: <div class='product'><h2>iPad Pro 12.9</h2><span class='price'>$1099</span><span class='category'>ta...
✅ Output: {
  "name": "iPad Pro 12.9",
  "price": "$1099",
  "category": "tablets",
  "manufacturer": "Apple"
}
----------------------------------------------------------------------------------------------------

Test 2: Missing Price
Expected: Can it handle missing price?
Input HTML: <div class='product'><h2>iPad Pro 12.9</h2><span class='category'>tablets</span><span class='brand'>...
✅ Output: {
  "name": "iPad Pro 12.9",
  "category": "tablets",
  "manufacturer": "Apple"
}
----------------------------------------------------------------------------------------------------

Test 3: Missing Brand
Expected: Can it handle missing brand?
Input HTML: <div class='product'><h2>Gaming Mouse</h2><span class='price'>$79</span><span class='catego

In [ ]:
FastLanguageModel.for_inference(model)

# Test with FAKE products that don't exist
hallucination_tests = [
    {
        "name": "Test: Fake Product with Only Name",
        "html": """<div class='product'><h2>Quantum Blaster 9000</h2></div>""",
        "reality": "This product doesn't exist - will it invent price/brand?"
    },
    {
        "name": "Test: Real Product, Wrong Price",
        "html": """<div class='product'><h2>iPhone 15 Pro</h2><span class='price'>$1</span><span class='category'>smartphones</span><span class='brand'>Apple</span></div>""",
        "reality": "Will it use the $1 from HTML or 'correct' it to real price?"
    },
    {
        "name": "Test: Real Product, Wrong Brand",
        "html": """<div class='product'><h2>MacBook Pro</h2><span class='price'>$999</span><span class='category'>laptops</span><span class='brand'>Microsoft</span></div>""",
        "reality": "Will it accept Microsoft or 'fix' it to Apple?"
    },
    {
        "name": "Test: Gibberish Product",
        "html": """<div class='product'><h2>Zxqwerty Foobar 3000</h2><span class='price'>$999</span></div>""",
        "reality": "What category/brand will it invent?"
    },
]

print("🔍 HALLUCINATION DETECTION TEST")
print("=" * 100)

for test in hallucination_tests:
    print(f"\n{test['name']}")
    print(f"Reality Check: {test['reality']}")
    print(f"Input: {test['html']}")

    prompt = f"Extract the product information:\n{test['html']}"
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.1)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract JSON
    start = response.rfind('{')
    end = response.rfind('}') + 1
    json_str = response[start:end] if start != -1 else None

    if json_str:
        try:
            parsed = json.loads(json_str)
            print(f"\n🤖 Model Output:")
            print(json.dumps(parsed, indent=2))

            # Check for hallucination
            if 'price' in parsed and 'price' not in test['html']:
                print("⚠️  HALLUCINATION DETECTED: Invented price!")
            if 'category' in parsed and 'category' not in test['html']:
                print("⚠️  HALLUCINATION DETECTED: Invented category!")
            if 'manufacturer' in parsed and 'brand' not in test['html']:
                print("⚠️  HALLUCINATION DETECTED: Invented manufacturer!")

        except Exception as e:
            print(f"❌ Parse error: {e}")

    print("-" * 100)

🔍 HALLUCINATION DETECTION TEST

Test: Fake Product with Only Name
Reality Check: This product doesn't exist - will it invent price/brand?
Input: <div class='product'><h2>Quantum Blaster 9000</h2></div>

🤖 Model Output:
{
  "name": "Quantum Blaster 9000",
  "price": "$1002",
  "category": "electronics",
  "brand": "Google"
}
⚠️  HALLUCINATION DETECTED: Invented price!
⚠️  HALLUCINATION DETECTED: Invented category!
----------------------------------------------------------------------------------------------------

Test: Real Product, Wrong Price
Reality Check: Will it use the $1 from HTML or 'correct' it to real price?
Input: <div class='product'><h2>iPhone 15 Pro</h2><span class='price'>$1</span><span class='category'>smartphones</span><span class='brand'>Apple</span></div>

🤖 Model Output:
{
  "name": "iPhone 15 Pro",
  "price": "$1",
  "category": "smartphones",
  "manufacturer": "Apple"
}
-----------------------------------------------------------------------------------------------

In [ ]:
# Test if it's memorizing specific prices
test_memorization = [
    # Does it remember $1002 is linked to specific products?
    """<div class='product'><h2>Random Gadget Alpha</h2><span class='price'>$1002</span></div>""",

    # Will it invent $1002 for tech products?
    """<div class='product'><h2>Tech Widget Pro</h2><span class='category'>electronics</span></div>""",
]

for html in test_memorization:
    prompt = f"Extract the product information:\n{html}"
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.1)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    start = response.rfind('{')
    end = response.rfind('}') + 1
    json_str = response[start:end]

    print(f"Input: {html}")
    print(f"Output: {json_str}\n")

Input: <div class='product'><h2>Random Gadget Alpha</h2><span class='price'>$1002</span></div>
Output: {"name": "Random Gadget Alpha", "price": "$1002"}

Input: <div class='product'><h2>Tech Widget Pro</h2><span class='category'>electronics</span></div>
Output: {"name": "Tech Widget Pro", "price": "$1002", "category": "electronics"}



In [ ]:
FastLanguageModel.for_inference(model)

# Test 1: Give ONLY the product name (to see if it remembers from training)
test1 = """<div class='product'><h2>Google Pixel 7</h2></div>"""

# Test 2: Give name + price (to see if it extracts correctly)
test2 = """<div class='product'><h2>Google Pixel 7</h2><span class='price'>$599</span></div>"""

# Test 3: Give complete data
test3 = """<div class='product'><h2>Google Pixel 7</h2><span class='price'>$712</span><span class='category'>smartphones</span><span class='brand'>Dell</span></div>"""

tests = [test1, test2, test3]
test_names = ["Only Name", "Name + Price", "Complete Data"]

for i, html in enumerate(tests):
    print(f"\n{'='*80}")
    print(f"Test {i+1}: {test_names[i]}")
    print(f"{'='*80}")
    print(f"Input HTML:\n{html}\n")

    prompt = f"Extract the product information:\n{html}"
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.1)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract JSON
    start = response.rfind('{')
    end = response.rfind('}') + 1
    json_str = response[start:end] if start != -1 else "No JSON found"

    print(f"Model Output:\n{json_str}\n")

    # Check if it hallucinated
    if i == 0:  # Test 1 - only name provided
        try:
            import json
            output = json.loads(json_str)
            if 'price' in output:
                print(f"⚠️  HALLUCINATION: Invented price = {output['price']}")
                print(f"   (Training data had: $712 and $2429 for Google Pixel 7)")
        except:
            pass


Test 1: Only Name
Input HTML:
<div class='product'><h2>Google Pixel 7</h2></div>

Model Output:
{"name": "Google Pixel 7", "price": "$1027", "category": "cameras", "brand": "Google"}

⚠️  HALLUCINATION: Invented price = $1027
   (Training data had: $712 and $2429 for Google Pixel 7)

Test 2: Name + Price
Input HTML:
<div class='product'><h2>Google Pixel 7</h2><span class='price'>$599</span></div>

Model Output:
{"name": "Google Pixel 7", "price": "$599"}


Test 3: Complete Data
Input HTML:
<div class='product'><h2>Google Pixel 7</h2><span class='price'>$712</span><span class='category'>smartphones</span><span class='brand'>Dell</span></div>

Model Output:
{"name": "Google Pixel 7", "price": "$712", "category": "smartphones", "manufacturer": "Dell"}



In [ ]:
# Test: Does structure matter?
test_cases = [
    {
        "name": "No structure - just name",
        "html": """<div class='product'><h2>Google Pixel 7</h2></div>"""
    },
    {
        "name": "Empty price span",
        "html": """<div class='product'><h2>Google Pixel 7</h2><span class='price'></span></div>"""
    },
    {
        "name": "Price span with text 'Unknown'",
        "html": """<div class='product'><h2>Google Pixel 7</h2><span class='price'>Unknown</span></div>"""
    },
]

for test in test_cases:
    print(f"\n{'='*80}")
    print(f"{test['name']}")
    print(f"{'='*80}")
    print(f"HTML: {test['html']}\n")

    prompt = f"Extract the product information:\n{test['html']}"
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.1)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    start = response.rfind('{')
    end = response.rfind('}') + 1
    json_str = response[start:end]

    print(f"Output: {json_str}\n")


No structure - just name
HTML: <div class='product'><h2>Google Pixel 7</h2></div>

Output: {"name": "Google Pixel 7", "price": "$1027", "category": "cameras", "brand": "Google"}


Empty price span
HTML: <div class='product'><h2>Google Pixel 7</h2><span class='price'></span></div>

Output: {"name": "Google Pixel 7", "price": "$1027"}


Price span with text 'Unknown'
HTML: <div class='product'><h2>Google Pixel 7</h2><span class='price'>Unknown</span></div>

Output: {"name": "Google Pixel 7", "price": "$1002"}



In [ ]:
  {
    "input": "Extract the product information:\n<div class='product'><h2>Google Pixel 7</h2><span class='price'>$299</span><span class='category'>cameras</span><span class='brand'>Google</span></div>",
    "output": {
      "name": "Google Pixel 7",
      "price": "$299",
      "category": "cameras",
      "manufacturer": "Google"
    }
  },
  {
    "input": "Extract the product information:\n<div class='product'><h2>Google Pixel 7</h2><span class='price'>$585</span><span class='category'>audio</span><span class='brand'>Google</span></div>",
    "output": {
      "name": "Google Pixel 7",
      "price": "$585",
      "category": "audio",
      "manufacturer": "Google"
    }
  },
    {
    "input": "Extract the product information:\n<div class='product'><h2>Google Pixel 7</h2><span class='price'>$534</span><span class='category'>smartphones</span><span class='brand'>Amazon</span></div>",
    "output": {
      "name": "Google Pixel 7",
      "price": "$534",
      "category": "smartphones",
      "manufacturer": "Amazon"
    }
  },
    {
    "input": "Extract the product information:\n<div class='product'><h2>Google Pixel 7</h2><span class='price'>$585</span><span class='category'>audio</span><span class='brand'>Asus</span></div>",
    "output": {
      "name": "Google Pixel 7",
      "price": "$585",
      "category": "audio",
      "manufacturer": "Asus"
    }
  },

    {
    "input": "Extract the product information:\n<div class='product'><h2>Google Pixel 7</h2><span class='price'>$712</span><span class='category'>smartphones</span><span class='brand'>Dell</span></div>",
    "output": {
      "name": "Google Pixel 7",
      "price": "$712",
      "category": "smartphones",
      "manufacturer": "Dell"
    }
  },

({'input': "Extract the product information:\n<div class='product'><h2>Google Pixel 7</h2><span class='price'>$299</span><span class='category'>cameras</span><span class='brand'>Google</span></div>",
  'output': {'name': 'Google Pixel 7',
   'price': '$299',
   'category': 'cameras',
   'manufacturer': 'Google'}},)